In [88]:
####Building a decision tree in Python####
import math
from collections import Counter, defaultdict
from functools import partial
from itertools import izip
from __future__ import division


#creating a list of tuples (dictionary + label) that will serve as "data" for the decision tree algorithm:

#attributes: 
#genre: What genre of game? (RPG, FPS, Action)
#console: What console? (N64, SNES, PS2)
#multiplayer: Is the game multiplayer?
#3D: Is the game in a 3D space? (polygonal)

trainingData = [
        ({'genre':'RPG','console':'N64','multiplayer':'no','3D':'no'},   False),
        ({'genre':'RPG','console':'N64','multiplayer':'no','3D':'yes'},  False),
        ({'genre':'Action','console':'PS2','multiplayer':'no','3D':'no'},     True),
        ({'genre':'FPS','console':'PS2','multiplayer':'no','3D':'no'},  True),
        ({'genre':'FPS','console':'SNES','multiplayer':'yes','3D':'no'},      True),
        ({'genre':'RPG','console':'SNES','multiplayer':'yes','3D':'no'},    False),
        ({'genre':'Action','console':'PS2','multiplayer':'no','3D':'yes'},    True),
        ({'genre':'Action','console':'N64','multiplayer':'yes','3D':'no'},      True),
        ({'genre':'FPS','console':'PS2','multiplayer':'no','3D':'yes'},False),
        ({'genre':'RPG','console':'N64','multiplayer':'yes','3D':'yes'},    True),
        ({'genre':'RPG','console':'PS2','multiplayer':'no','3D':'yes'},    True),
        ({'genre':'Action','console':'N64','multiplayer':'no','3D':'yes'},    False)
    ]

testingData = [
        {'genre':'Action','console':'PS2','multiplayer':'yes','3D':'no'},
        {'genre':'FPS','console':'PS2','multiplayer':'yes','3D':'no'},
        {'genre':'RPG','console':'SNES','multiplayer':'no','3D':'yes'},
        {'genre':'Action','console':'N64','multiplayer':'yes','3D':'yes'},
        {'genre':'RPG','console':'N64','multiplayer':'yes','3D':'no'},
        {'genre':'RPG','console':'PS2','multiplayer':'yes','3D':'no'},
        {'genre':'Action','console':'N64','multiplayer':'yes','3D':'yes'},
        {'genre':'RPG','console':'PS2','multiplayer':'no','3D':'no'},
        {'genre':'Action','console':'SNES','multiplayer':'no','3D':'yes'},
    ]

#Finding entropy across an entire data-set:
def entropy(labelProportions):
    sum = 0
    for p in labelProportions:
        #entropy for each proportion
        sum += -p*math.log(p,2)
    return sum

def datasetEntropy(data):
    #labels = []
    #for label in data:
       # temp = data[]
        #labels = data[label]
    labels = [label for _, label in data]
    proportions = labelProportions(labels)
    return entropy(proportions)

#labelProportions will return a list of proportions for each label in the set
def labelProportions(labels):
    partitions = []
    totalObservations = len(labels)
    #return [count/totalObservations for count in Counter(labels).values()]
    for count in Counter(labels).values():
        partitions.append(count/totalObservations)
    return partitions
        
#handles the partitioning
def partition(inputs, attribute):
    groups = defaultdict(list)
    for input in inputs:
        key =  input[0][attribute]
        #print key
        groups[key].append(input)
    return groups

#finds the entropy of the partition, using the entropy method created earlier    
def partitionEntropy(subsets):
    freq = 0
    totalCount = 0
    
    for subset in subsets:
        freq+=len(subset)
        
    for subset in subsets:
        totalCount += datasetEntropy(subset)*len(subset)/freq
    return totalCount

def partEntropyBuilder(inputs, attribute):
    partitions = partition(inputs, attribute)
    return partitionEntropy(partitions.values())


#####Builds the tree based on the calculations of entropy above#####

def treeBuilder(data, possSplit = None):
    #starting with any potential splits as a parameter - dont want it in the code for recursion purposes
    if possSplit is None:
        possSplit = data[0][0].keys()
       
    #Determine terminal leaf nodes of the tree by finding the number of binary labels
    #Only accounting for a binary tree in which the label can be one of two values
    totalCount = len(data)
    
    #count the frequency of one label 
       # for item, label in data:
           # if label:
            
    #labelTwoTotal = totalCount - labelOneTotal
    #List comprehension to find the total amount of the 'yes' or 'true' labels
    labelOneTotal = len([label for item, label in data if label])
    #print labelOneTotal
    #count number of other possible label
    labelTwoTotal = totalCount - labelOneTotal
    
    #Finally return a labelOne or labelTwo leaf node if they contain 100% of one label
    if labelOneTotal == 0:
        return False
    if labelTwoTotal == 0:
        return True
    
    #Return the most frequent label for a leaf node if there are no attributes left to split 
    #will only do this if the list of possible attributes to split on is empty. that means a leaf needs to be returned
    #labelOneTotal is the number of true statements, therefore it should be returning true if the total is greater 
    if not possSplit:
        if labelOneTotal > labelTwoTotal:
            return True
        else:
            return False  
    #max(labelOneTotal, labelTwoTotal)
    
    #was stuck on this portion for a while, but then I looked up a few examples and stumbled upon partials, which actually was a new concept to me
    #the partial function allows you to essentially 'fill' a function's parameter's before it is being called
    #here I'm  are a new function with less parameters than what it takes
    #split on lowest entroy by partition
    #getPartitionEntropy = partEntropyBuilder(data, possSplit)
    #print getPartitionEntropy
    
    #optimalAttri = min(possSplit, getPartitionEntropy)
    optimalAttri = min(possSplit, key = partial(partEntropyBuilder, data))
    
    sections = partition(data, optimalAttri)
    
    #Create new set of attributes that will can be evaluated for further split
    splitNewPool = [i for i in possSplit if i != optimalAttri]
    
    #build the rest of the subtrees
    #substrees will be dictionaries just like the general tree
    sTrees = {attriValue : treeBuilder(subset, splitNewPool) for attriValue, subset in sections.iteritems()}
    
    #required if the new data does not have the correct information that matches the tree
    sTrees[None] = labelOneTotal > labelTwoTotal
    
    return [optimalAttri, sTrees]
    
                    

In [89]:
videoGameTree = treeBuilder(trainingData)

In [90]:
print videoGameTree

['console', {'PS2': ['genre', {'Action': True, None: True, 'FPS': ['3D', {'yes': False, None: False, 'no': True}], 'RPG': True}], 'N64': ['multiplayer', {'yes': True, None: False, 'no': False}], None: True, 'SNES': ['genre', {None: False, 'FPS': True, 'RPG': False}]}]


In [91]:
#Need a way to determine what new data would be classified as (should I play it or not?)
#takes a tree (dictionary and test data (list of labels and dictionaries))
def findLabel(tree, newData):
    
    #start with the leaf case - if so return the value of the leaf and you're done finding the label
    if tree in [True, False]:
        return tree
    
    #start from the top of the tree and get the first attribute
    attri, subtreeDictionary = tree
    subKey = newData.get(attri)
    
    #handles none case
    if subKey not in subtreeDictionary:
        subKey = None
        
    #moves to the appropriate subtree
    subtree = subtreeDictionary.get(subKey)
    
    #back to searching the tree
    return findLabel(subtree, newData)
    
    

In [92]:
#seeing if this will come up with labels for unlabeled data called "testing data"
for item in testingData:
    print findLabel(videoGameTree, item)

True
True
False
True
True
True
True
True
False


In [112]:
i = iter(videoGameTree)
videoGameDictTree = dict(izip(i,i))
print videoGameDictTree

{'console': {'PS2': ['genre', {'Action': True, None: True, 'FPS': ['3D', {'yes': False, None: False, 'no': True}], 'RPG': True}], 'N64': ['multiplayer', {'yes': True, None: False, 'no': False}], None: True, 'SNES': ['genre', {None: False, 'FPS': True, 'RPG': False}]}}
<type 'dict'>
['console', {'PS2': ['genre', {'Action': True, None: True, 'FPS': ['3D', {'yes': False, None: False, 'no': True}], 'RPG': True}], 'N64': ['multiplayer', {'yes': True, None: False, 'no': False}], None: True, 'SNES': ['genre', {None: False, 'FPS': True, 'RPG': False}]}]


In [134]:
#def reduceTree(tree):
    #navigate to the bottom of the tree:
    #for key, node in tree.items():
        #if isinstance(node, dict):
            #tree[key] = reduceTree(node)
    #if some statement is true
        #remove subtree, replacing with new leaf value
    #return tree

In [135]:
#reduceTree(videoGameDictTree)

{'PS2': ['genre', {'Action': True, None: True, 'FPS': ['3D', {'yes': False, None: False, 'no': True}], 'RPG': True}], 'N64': ['multiplayer', {'yes': True, None: False, 'no': False}], None: True, 'SNES': ['genre', {None: False, 'FPS': True, 'RPG': False}]}
{'console': None}
